In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp

# T_SB = np.array([[0,  1,   0, 0],
#                  [0,  0,  -1, -(0.420+0.025+0.080)],
#                  [-1, 0,   0, 0.89],
#                  [0,  0,   0, 1]])


In [9]:
# Testing if we get 4 configurations of theta 1,2,3 that results in the same wrist position

theta = np.pi/180*np.array([-45, -90, 90, 73, 0  , 17])

kr6 = KR6(theta)

P_W = tools.pointFromT(kr6.Tsw)

thetalists = np.around(kr6.aIK_thetaLists,2) #creating the 4 sets of thetas that should provide the same wrist position as the original input of KR6

np.around(thetalists*180/np.pi,1)

for i in thetalists:
    print(np.around(i*180/np.pi,2))
    i = np.append(i,0)
    i = np.append(i,0)
    i = np.append(i,0)
    kr6_test = KR6(i)
    P_W_test = tools.pointFromT(kr6_test.Tsw)
    print(np.allclose(P_W, P_W_test, 0.02))


[-45.26  -8.59 -80.21]
True
[-45.26 -89.95  89.95]
True
[ 135.22 -171.31   83.08]
True
[135.22 -96.26 -73.91]
True


In [11]:

S = kr6._Slist

M = kr6._M

T = kr6.Tsb_poe

print(T)

R_46, R_46sp = tools.inverseKinematicsTheta456(thetalists, T, S, M)

R_46

print( np.arctan2( float(R_46[0,1]) , float(R_46[0,2])) * 180/np.pi - 90)

print(np.arctan2(np.sqrt(float(R_46[0,1])**2 + float(R_46[0,2])),float(R_46[2,1])*180/np.pi) * 180/np.pi - 180)

print( np.arctan2(-float(R_46[2,1]) , float(R_46[2,0])) * 180/np.pi)



[[-7.07106781e-01  7.97859931e-18  7.07106781e-01  3.71231060e-01]
 [ 7.07106781e-01 -7.97859931e-18  7.07106781e-01  3.71231060e-01]
 [-1.12834434e-17  1.00000000e+00  0.00000000e+00  8.90000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
-90.0
-0.06783668668154519
90.0


In [31]:

# th4, th5, th6 = sp.symbols("theta_4 theta_5 theta_6")

# def rotX_sp(th):
#     x = sp.Matrix([[1, 0, 0],
#                [0, sp.cos(th), -sp.sin(th)],
#                [0, sp.sin(th), sp.cos(th)]])
#     return x

# def rotZ_sp(th):
#     z = sp.Matrix([[sp.cos(th), -sp.sin(th), 0],
#                [sp.sin(th), sp.cos(th), 0],
#                [0, 0, 0]])
#     return z

# def rotxyx(th4,th5,th6):
    th4 = -th4
    th6 = -th6
    # c4, s4 = sp.cos(th4), sp.sin(th4)
    # c5, s5 = sp.cos(th5), sp.sin(th5)
    # c6, s6 = sp.cos(th6), sp.sin(th6)

    # x = sp.Matrix([[1, 0, 0],
    #            [0, sp.cos(th4), -sp.sin(th4)],
    #            [0, sp.sin(th4), sp.cos(th4)]])

    # y = sp.Matrix([[sp.cos(th5), 0, sp.sin(th5)],
    #            [0, 0, 0],
    #            [-sp.sin(th5), 0, sp.cos(th5)]])

    # x2 = sp.Matrix([[1, 0, 0],
    #            [0, sp.cos(th6), -sp.sin(th6)],
    #            [0, sp.sin(th6), sp.cos(th6)]])

    # z = sp.Matrix([[sp.cos(th6), -sp.sin(th6), 0],
    #            [sp.sin(th6), sp.cos(th6), 0],
    #            [0, 0, 0]])
    
    R_45 = sp.Matrix([[sp.cos(th4), -sp.sin(th4), 0],
               [sp.sin(th4), sp.cos(th4), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(-PI/2), -sp.sin(-PI/2)],
               [0, sp.sin(-PI/2), sp.cos(-PI/2)]])

    R_56 = sp.Matrix([[sp.cos(th5), -sp.sin(th5), 0],
               [sp.sin(th5), sp.cos(th5), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(PI/2), -sp.sin(PI/2)],
               [0, sp.sin(PI/2), sp.cos(PI/2)]])


    R_6B = sp.Matrix([[sp.cos(th6), -sp.sin(th6), 0],
               [sp.sin(th6), sp.cos(th6), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(PI), -sp.sin(PI)],
               [0, sp.sin(PI), sp.cos(PI)]])
    # rot_youtube = sp.Matrix([[c4*s5*c6 - s4*s6, -c4*s5*s6 - s4*c6, -c4*c5],
    #                [s4*s5*c6 + c4*s6, -s4*s5*s6 + c4*c6, -s4*c5],
    #                [c5*c6, -c5*s6, s5]])

    # x = sp.Matrix([[, , ],
    #                [, , c4],
    #                [, , ]])
    # x = sp.Matrix([[1, 0, 0],
    #                [0, 0, 0],
    #                [0, 0, 0]])
    # x = sp.Matrix([[1, 0, 0],
    #                [0, 0, 0],
    #                [0, 0, 0]])
    return R_45,R_56,R_6B

# print(np.around(R_4B,2))
# print(np.arccos(-1))
# PI = np.pi
# R_45 = rotZ_sp(th4) @ rotX_sp(-PI/2)
# R_56 = rotZ_sp(th5) @ rotX_sp(PI/2) 
# R_6B = rotZ_sp(th6) @ rotX_sp(PI)
# R_45.evalf(1)*R_56.evalf(1)*R_6B.evalf(1)


C:\Users\PC\anaconda3\envs\tpk4170_venv\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


array([  0., -90.,  90.,   0.,   0.,   0.])

In [ ]:
# config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])
# config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
# config3 = np.array([0, np.pi/2, 0, 0, 0, 0])





# # kr6.add_FK_to_viewer(viewer)

# # print(f"Number of iteration: {kr6.IK[3]}")

# kr6.add_nIK_to_viewer(viewer)
viewer = Viewer(background="white")
viewer.add(Grid())
KR6(theta).add_FK_to_viewer(viewer)
